# 6.2 勾配消失とLSTM

- RNNには勾配消失という問題点あり<br>
↓<br>
- ゲート付きRNN
    - 代表的なモデル
        - LSTM ⇦この節で取り上げ
        - GTU etc

## 6.2.1 LSTMのインターフェース

- RNN記法を下図のように簡略化
- $ tanh(h_{t-1}W_h + x_tW_x + b) $ という計算を「tanh」と表す

![fig6](fig6/6_10.png)

- RNNレイヤとLSTMレイヤの比較(下図)
    - LSTMにはc(記憶セル)あり
        - 記憶セルの特徴
            - 自分自身だけで(LSTMレイヤ内だけで)データの受け渡し・他レイヤへの出力はしない

![fig6](fig6/6_11.png)

## 6.2.2 LSTMレイヤの組み立て

- 現在の記憶セル$c_t$は($c_{t-1}, h_{t-1}, x_t$)からの計算によって算出
- ポイント
    - 更新された$c_t$を使って、隠れ状態の$h_t$が計算される
        - $h_t = tanh(c_t)$

![fig6](fig6/6_12.png)

- LSTMにおける「ゲート」の機能
    - どれだけゲートを開くか、水を次へ流すかをコントロール
        - 開き具合は、0.0~1.0 までの実数で表される
        - ゲートの開き具合のコントロールの為に、専用の重みパラメータが用いられる

![6_14](fig6/6_14.png)

## 6.2.3 outputゲート

- outputゲート
    - $tanh(c_t)$ の各要素に対して「それらが次時刻の隠れ状態($h_t$)としてどれだけ重要か」を調整
    - 開き具合(次へ何%だけ通すか)は、入力$x_t$と前の状態$h_{t-1}$から求める
    - ここで使用する重みパラメータやバイアスの上付き文字に$o$(outputの頭文字)を使用
    - sigmoind関数はα()で表す
    ![6_1](fig6/6_1.png)
    - $o$ と $tanh(c_t)$の要素ごとの積を$h_t$として出力
    ![6_15](fig6/6_15.png)
    - outputゲートで行う式の計算を「α」で表す
    - $h_t$は$o$と$tanh(c_t)$の積によって計算
        - アダマール積
        ![6_2](fig6/6_2.png)

## 6.2.4 forgetゲート

- 記憶セル($c$)に対して「何を忘れるか」を明示的に指示
- ![6_16](fig6/6_16.png)
- ![6_3](fig6/6_3.png)
    - 式(6.3)によってforgetゲートの出力fが求められる
    - このfと前の記憶セルである$c_{t-1}$との要素ごとの積($c_t = f⊙c_{t-1}$)によって$c_{t}$が求められる

## 6.2.5 新しい記憶セル

- 新しく覚えるべき情報を記憶セルに追加する必要あり(現状は忘れる機能のみ)
- 下図のようにtanhノードを新たに追加
![6_17](fig6/6_17.png)
- tanhノードによって計算された結果が前時刻の記憶セル$c_{t-1}$に加算される
- このノードは「ゲート」ではなく新しい「情報」を記憶セルに追加することが目的
![6_4](fig6/6_4.png)
- このgが前時刻の$c_{t-1}$に加算されることで新しい記憶が生まれる

## 6.2.6 inputゲート

- gに対してゲートを加える
![6_18](fig6/6_18.png)
- gの各要素が新たに追加する情報としてどれだけ価値があるかを判断
- inputゲートによって重みづけされた情報が新たに追加される
- inputゲートを「α」で表し、その出力をiとする
![6_5](fig6/6_5.png)
- iとgの要素ごとの積の結果を記憶せるに追加

## 6.2.7 LSTMの勾配の流れ

- LSTMが勾配消失を起こさない理由
    - →記憶セルcの逆伝播に注目すると見えてくる

![6_19](fig6/6_19.png)

- 記憶セルにフォーカス
    - 「+」ノード・・・上流から伝わる勾配をそのまま流すだけ→勾配の変化(劣化)は無し
    - 「×」ノード・・・「要素ごとの積(アダマール積)」
        - RNNの逆伝播では行列の積を繰り返し行ってきたので勾配消失・爆発が発生
        - LSTMの逆伝播では、毎時刻異なるゲート値によって要素ごとの積を計算<br>→勾配消失を起こさない理由
        - 「×」ノードの計算はforgetゲートによってコントロール
            - 「忘れるべき」と導いた勾配の要素は小さく
            - 「忘れてはいけない」と導いた勾配の要素は劣化することなく過去方向へ伝わる

# 6.3 LSTMの実装

- 最初に、1ステップの処理をLSTMクラスとして実装
- そして、Tステップ分をまとめて処理するクラスをTimeLSTMとして実装
-　LSTMで行う計算は下図 <br>
![6_6](fig6/6_6.png)<br>
- 式(6.6)の4つのアフィン変換※がポイント(※ $xW_x + hW_h + b$のような式)
        - 一つの式にまとめて計算可能
![6_20](fig6/6_20.png)<br>

- 本来であれば4回個別に行っていたアフィン変換の計算を1回の計算で済ませることが可能に<br>→計算の高速化
- $W_x, W_h, b$　にそれぞれ4つ分の重みが含まれていると仮定して、LSTMの計算グラフを図示<br>
![6_21](fig6/6_21.png)
- ここでは初めに4つ分のアフィン変換をまとめて実施
-　そして、sliceノードによって、その4つの結果を取り出す
    - sliceノード・・・アフィン変換の結果を4分割して取り出すだけの単純なノード

◆LSTMクラスの初期化コード
```python
class LSTM:
    def __init__(self, Wx, Wh, b):
        '''

        Parameters
        ----------
        Wx: 入力`x`用の重みパラーメタ（4つ分の重みをまとめる）
        Wh: 隠れ状態`h`用の重みパラメータ（4つ分の重みをまとめる）
        b: バイアス（4つ分のバイアスをまとめる）
        '''
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None
```

- 初期化の引数は、重みパラメータのWx, Wh, b
    - 重みには4つ分の重みが纏められている
    - パラメータはメンバ変数のparamsに設定・初期化
    - cacheは順伝播での中間結果を保持するために使用。逆伝播の計算でも使用。

◆順伝播の実装<br>
- 引数は、現時刻の入力x、前時刻の隠れ状態h_prev、前時刻の記憶セルc_prev

```python
    def forward(self, x, h_prev, c_prev):
        Wx, Wh, b = self.params
        N, H = h_prev.shape

        A = np.dot(x, Wx) + np.dot(h_prev, Wh) + b

        f = A[:, :H]
        g = A[:, H:2*H]
        i = A[:, 2*H:3*H]
        o = A[:, 3*H:]

        f = sigmoid(f)
        g = np.tanh(g)
        i = sigmoid(i)
        o = sigmoid(o)

        c_next = f * c_prev + g * i
        h_next = o * np.tanh(c_next)

        self.cache = (x, h_prev, c_prev, i, f, g, o, c_next)
        return h_next, c_next
```
- まず初めにアフィン変換が行われる
 - メンバ変数のWx、Wh、bには4つ分のパラメータが格納。形状は以下の通り
 ![6_22](fig6/6_22.png)
- バッチ数をN、入力データの次元数をD、記憶セルと隠れ状態の次元数を両者ともH
- 計算結果のA には4 つ分のアフィン変換の結果が格納
- そこからA[:, :H] やA[:, H:2*H] のようにスライスして取り出すことで、それ以降の演算ノードへ分配

◆LSTMの逆伝播
- 逆伝播では、4つの勾配を結合する必要あり
 ![6_23](fig6/6_23.png)
 
 
```python
    def backward(self, dh_next, dc_next):
        Wx, Wh, b = self.params
        x, h_prev, c_prev, i, f, g, o, c_next = self.cache

        tanh_c_next = np.tanh(c_next)

        ds = dc_next + (dh_next * o) * (1 - tanh_c_next ** 2)

        dc_prev = ds * f

        di = ds * g
        df = ds * c_prev
        do = dh_next * tanh_c_next
        dg = ds * i

        di *= i * (1 - i)
        df *= f * (1 - f)
        do *= o * (1 - o)
        dg *= (1 - g ** 2)

        dA = np.hstack((df, dg, di, do))

        dWh = np.dot(h_prev.T, dA)
        dWx = np.dot(x.T, dA)
        db = dA.sum(axis=0)

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        dx = np.dot(dA, Wx.T)
        dh_prev = np.dot(dA, Wh.T)

        return dx, dh_prev, dc_prev
```
- 4つの勾配(df, dg, di, do)を連結してdAを作成
    - np.hstack()　が使用可能<br>引数に与えられた配列を横方向に連結

## 6.3.1 TimeLSTMの実装

- T個分の時系列データをまとめて処理するレイヤ
![6_24](fig6/6_24.png)
- RNNで学習を行う際には、Truncated BPTTを行う
    - 逆伝播のつながりを適当な長さで断ち切る
![6_25](fig6/6_25.png)

◆Time LSTMの実装
- LSTM では隠れ状態のh に加えて記憶セルc も用いますが、TimeLSTM クラスの実装はTimeRNN クラスの場合とほとんど同じです。ここでも引数のstateful によって状態を維持するかどうかを指定します。

```python
class TimeLSTM:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None

        self.h, self.c = None, None
        self.dh = None
        self.stateful = stateful

    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape
        H = Wh.shape[0]

        self.layers = []
        hs = np.empty((N, T, H), dtype='f')

        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')
        if not self.stateful or self.c is None:
            self.c = np.zeros((N, H), dtype='f')

        for t in range(T):
            layer = LSTM(*self.params)
            self.h, self.c = layer.forward(xs[:, t, :], self.h, self.c)
            hs[:, t, :] = self.h

            self.layers.append(layer)

        return hs

    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D = Wx.shape[0]

        dxs = np.empty((N, T, D), dtype='f')
        dh, dc = 0, 0

        grads = [0, 0, 0]
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh, dc = layer.backward(dhs[:, t, :] + dh, dc)
            dxs[:, t, :] = dx
            for i, grad in enumerate(layer.grads):
                grads[i] += grad

        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh
        return dxs

    def set_state(self, h, c=None):
        self.h, self.c = h, c

    def reset_state(self):
        self.h, self.c = None, None
```

# 6.4 LSTMを使った言語モデル


![6_26](fig6/6_26.png)

- Rnnlmというクラスで実装

In [1]:
# coding: utf-8
import sys
sys.path.append('..')
from common.time_layers import *
from common.base_model import BaseModel


class Rnnlm(BaseModel):
    def __init__(self, vocab_size=10000, wordvec_size=100, hidden_size=100):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        # 重みの初期化
        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')

        # レイヤの生成
        self.layers = [
            TimeEmbedding(embed_W),
            TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True),
            TimeAffine(affine_W, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.lstm_layer = self.layers[1]

        # すべての重みと勾配をリストにまとめる
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

    def predict(self, xs):
        for layer in self.layers:
            xs = layer.forward(xs)
        return xs

    def forward(self, xs, ts):
        score = self.predict(xs)
        loss = self.loss_layer.forward(score, ts)
        return loss

    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout

    def reset_state(self):
        self.lstm_layer.reset_state()


In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
from common.optimizer import SGD
from common.trainer import RnnlmTrainer
from common.util import eval_perplexity
from dataset import ptb
from rnnlm import Rnnlm


# ハイパーパラメータの設定
batch_size = 20
wordvec_size = 100
hidden_size = 100  # RNNの隠れ状態ベクトルの要素数
time_size = 35  # RNNを展開するサイズ
lr = 20.0
max_epoch = 4
max_grad = 0.25

# 学習データの読み込み
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_test, _, _ = ptb.load_data('test')
vocab_size = len(word_to_id)
xs = corpus[:-1]
ts = corpus[1:]

# モデルの生成
model = Rnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)

# 勾配クリッピングを適用して学習
trainer.fit(xs, ts, max_epoch, batch_size, time_size, max_grad,
            eval_interval=20)
trainer.plot(ylim=(0, 500))

# テストデータで評価
model.reset_state()
ppl_test = eval_perplexity(model, corpus_test)
print('test perplexity: ', ppl_test)

# パラメータの保存
model.save_params()


| epoch 1 |  iter 1 / 1327 | time 0[s] | perplexity 10000.47
| epoch 1 |  iter 21 / 1327 | time 4[s] | perplexity 3106.71
| epoch 1 |  iter 41 / 1327 | time 8[s] | perplexity 1241.52
| epoch 1 |  iter 61 / 1327 | time 12[s] | perplexity 981.25
| epoch 1 |  iter 81 / 1327 | time 16[s] | perplexity 805.74
| epoch 1 |  iter 101 / 1327 | time 20[s] | perplexity 654.71
| epoch 1 |  iter 121 / 1327 | time 24[s] | perplexity 659.23
| epoch 1 |  iter 141 / 1327 | time 29[s] | perplexity 606.01
| epoch 1 |  iter 161 / 1327 | time 33[s] | perplexity 577.84
| epoch 1 |  iter 181 / 1327 | time 37[s] | perplexity 573.54
| epoch 1 |  iter 201 / 1327 | time 41[s] | perplexity 496.37
| epoch 1 |  iter 221 / 1327 | time 46[s] | perplexity 485.98
| epoch 1 |  iter 241 / 1327 | time 50[s] | perplexity 442.10
| epoch 1 |  iter 261 / 1327 | time 55[s] | perplexity 463.54
| epoch 1 |  iter 281 / 1327 | time 59[s] | perplexity 450.37
| epoch 1 |  iter 301 / 1327 | time 64[s] | perplexity 390.10
| epoch 1 |  